In [70]:
# Execution of this entire notebook should take around 4-8 minutes
# All the imports

import numpy as np
#import itk
import vtk
import itkwidgets
import math

import sys
sys.path.append('/data/ITKPR22/ITK/ITK-build/Wrapping/Generators/Python')

import itkConfig
itkConfig.LazyLoading = False
import itk

In [71]:
def readvtk(filename):
    a = vtk.vtkPolyDataReader()
    a.SetFileName(filename)
    a.Update()
    m1 = a.GetOutput()
    return m1

In [72]:
# Fetch the files

FIXED_MESH_FILE  = r'data/A_J_skull_.ply'
MOVING_MESH_FILE = r'data/Ark_J_skull_.ply'

# FIXED_MESH_FILE  = r'data/129S1_SVIMJ_.ply'
# MOVING_MESH_FILE = r'data/129X1_SVJ_.ply'

paths = [FIXED_MESH_FILE, MOVING_MESH_FILE]

import os
import importlib
from urllib.request import urlretrieve

# Download meshes
os.makedirs('data',exist_ok=True)
if not os.path.exists(FIXED_MESH_FILE):
    #url = 'https://github.com/SlicerMorph/Mouse_Models/raw/main/Models/129S1_SVIMJ_.ply'
    url = 'https://github.com/SlicerMorph/Mouse_Models/raw/main/Models/A_J_skull_.ply'
    urlretrieve(url, FIXED_MESH_FILE)
if not os.path.exists(MOVING_MESH_FILE):
    #url = 'https://github.com/SlicerMorph/Mouse_Models/raw/main/Models/129X1_SVJ_.ply'
    url = 'https://github.com/SlicerMorph/Mouse_Models/raw/main/Models/Ark_J_skull_.ply'
    urlretrieve(url, MOVING_MESH_FILE)

In [73]:
# Clean the mehes and Triangulate them as TSD only works with Triangle Meshes

vtk_meshes = list()

for path in paths:
    reader = vtk.vtkPLYReader()
    reader.SetFileName(path)
    reader.Update()
        
    cleaner = vtk.vtkCleanPolyData()
    cleaner.SetInputData(reader.GetOutput())
    cleaner.ConvertLinesToPointsOn()
    cleaner.ConvertPolysToLinesOff()
    cleaner.SetTolerance(0.0)
    cleaner.Update()
    vtk_mesh = cleaner.GetOutput()

    triangle_filter = vtk.vtkTriangleFilter()
    triangle_filter.SetInputData(vtk_mesh)
    triangle_filter.SetPassLines(False)
    triangle_filter.SetPassVerts(False)
    triangle_filter.Update()
    vtk_mesh = triangle_filter.GetOutput()
    
    vtk_meshes.append(vtk_mesh)
    
# Write back out to a filetype supported by ITK
vtk_paths = [path.strip('.ply') + '.vtk' for path in paths]
for idx, mesh in enumerate(vtk_meshes):
    writer = vtk.vtkPolyDataWriter()
    writer.SetInputData(mesh)
    writer.SetFileVersion(42)
    writer.SetFileName(vtk_paths[idx])
    writer.Update()
    
itk_meshes = [itk.meshread(path, pixel_type=itk.D) for path in vtk_paths]

In [74]:
# Convert meshes to images for performing moment based initialization

itk_transforms = list()
itk_images = []

for mesh in itk_meshes:
    p = itk.array_from_vector_container(mesh.GetPoints())
    
    x = float(np.mean(p[:, 0]))
    y = float(np.mean(p[:, 1]))
    z = float(np.mean(p[:, 2]))
    
    #p[:, 0] = p[:, 0] - x
    #p[:, 1] = p[:, 1] - y
    #p[:, 2] = p[:, 2] - z
    
    mesh.SetPoints(itk.vector_container_from_array(p.flatten()))
    
    itk_image = itk.triangle_mesh_to_binary_image_filter(mesh,
                                                       origin=[0, 0, 0],
                                                       spacing=[0.25, 0.25, 0.25],
                                                       size=[100, 100, 100])
    
    itk_images.append(itk_image)
    
    calculator = itk.ImageMomentsCalculator[type(itk_image)].New()
    calculator.SetImage(itk_image)
    calculator.Compute()
    itk_transforms.append(calculator.GetPhysicalAxesToPrincipalAxesTransform())

In [75]:
# Write the Moment based initialized meshes as vtk file

itk_transformed_meshes = [
    itk.transform_mesh_filter(mesh, transform=itk_transforms[idx])
    for idx, mesh in enumerate(itk_meshes)
]


movingMesh = itk_transformed_meshes[0]
fixedMesh  = itk_transformed_meshes[1]

itk.meshwrite(movingMesh, 'movingMesh.vtk')
itk.meshwrite(fixedMesh, 'fixedMesh.vtk')

In [76]:
all_points1 = itk.array_from_vector_container(movingMesh.GetPoints())
all_points2 = itk.array_from_vector_container(fixedMesh.GetPoints())

a1 = all_points1.shape[0]
a2 = all_points2.shape[0]

number_of_rows = 3000

In [77]:
random_indices = np.random.choice(a1, size=number_of_rows)
p1 = all_points1[random_indices, :]

random_indices = np.random.choice(a1, size=number_of_rows)
p2 = all_points2[random_indices, :]

ps1 = itk.Mesh[itk.D, 3].New()
ps2 = itk.Mesh[itk.D, 3].New()

ps1.SetPoints(itk.vector_container_from_array(p1.flatten()))
ps2.SetPoints(itk.vector_container_from_array(p2.flatten()))

In [78]:
print(ps1.GetNumberOfPoints())

3000


In [79]:
PointSetType  = type(ps1)

TransformType = itk.Euler3DTransform[itk.D]
transform = TransformType.New()
transform.SetIdentity()

MetricType = itk.EuclideanDistancePointSetToPointSetMetricv4.PSD3.New()
metric = MetricType.New()
metric.SetMovingPointSet(ps2)
metric.SetFixedPointSet(ps1)
metric.SetMovingTransform(transform)
metric.Initialize()
#print(metric.GetValue())

def print_iteration():
    print(
        f"It: {optimizer.GetCurrentIteration()}"
        f" metric value: {optimizer.GetCurrentMetricValue():.6f} "
    )

optimizer = itk.ConjugateGradientLineSearchOptimizerv4Template[itk.D].New()
optimizer.SetNumberOfIterations( 50 )
#optimizer.SetScalesEstimator( shiftScaleEstimator )
optimizer.SetMaximumStepSizeInPhysicalUnits( 0.1 )
optimizer.SetMinimumConvergenceValue( 0.0 )
optimizer.SetConvergenceWindowSize( 50 )
optimizer.SetMetric(metric)
optimizer.AddObserver(itk.IterationEvent(), print_iteration)

print('Before > ', metric.GetValue())

optimizer.StartOptimization()

print('After > ', optimizer.GetCurrentMetricValue())

Before >  1.3451792651442234
It: 0 metric value: 1.345179 
It: 1 metric value: 0.771722 
It: 2 metric value: 0.747973 
It: 3 metric value: 0.746125 
It: 4 metric value: 0.745895 
It: 5 metric value: 0.745850 
It: 6 metric value: 0.745843 
It: 7 metric value: 0.745842 
It: 8 metric value: 0.745842 
It: 9 metric value: 0.745842 
It: 10 metric value: 0.745842 
It: 11 metric value: 0.745842 
It: 12 metric value: 0.745842 
It: 13 metric value: 0.745842 
It: 14 metric value: 0.745842 
It: 15 metric value: 0.745842 
It: 16 metric value: 0.745842 
It: 17 metric value: 0.745842 
It: 18 metric value: 0.745842 
It: 19 metric value: 0.745842 
It: 20 metric value: 0.745842 
It: 21 metric value: 0.745842 
It: 22 metric value: 0.745842 
It: 23 metric value: 0.745842 
It: 24 metric value: 0.745842 
It: 25 metric value: 0.745842 
It: 26 metric value: 0.745842 
It: 27 metric value: 0.745842 
It: 28 metric value: 0.745842 
It: 29 metric value: 0.745842 
It: 30 metric value: 0.745842 
It: 31 metric value:

In [27]:
from vtk.util import numpy_support

m1 = readvtk('movingMesh.vtk')
m2 = readvtk('fixedMesh.vtk')

print(m1.GetNumberOfPoints(), m2.GetNumberOfPoints())

all_points1 = numpy_support.vtk_to_numpy(m1.GetPoints().GetData())
all_points2 = numpy_support.vtk_to_numpy(m2.GetPoints().GetData())

a1 = all_points1.shape[0]
a2 = all_points2.shape[0]

print(all_points1.shape, all_points2.shape)

number_of_rows = 3000
best_transform = None
best_value     = 10000000

icp = vtk.vtkIterativeClosestPointTransform()
icp.DebugOn()
icp.SetDebug(True)
icp.GetLandmarkTransform().SetModeToRigidBody()
icp.CheckMeanDistanceOn()
icp.SetMaximumNumberOfLandmarks(200)
icp.SetMaximumNumberOfIterations(10)
icp.SetMaximumMeanDistance(0.00001)
icp.StartByMatchingCentroidsOn()
    
for i in range(10):
    random_indices = np.random.choice(a1, 
                                  size=number_of_rows)
    p1 = all_points1[random_indices, :]
    
    random_indices = np.random.choice(a1, 
                                  size=number_of_rows)
    p2 = all_points2[random_indices, :]
    
    vp1 = vtk.vtkPointSet()
    vp2 = vtk.vtkPointSet()
    
    points1 = vtk.vtkPoints()
    points2 = vtk.vtkPoints()
    
    points1.SetData(numpy_support.numpy_to_vtk(p1))
    points2.SetData(numpy_support.numpy_to_vtk(p2))
    
    vp1.SetPoints(points1)
    vp2.SetPoints(points2)
    
    print('Points Created')
#     icp.SetSource(vp1)
#     icp.SetTarget(vp2)
#     icp.Modified()
    icp.Update()
    
    print('ICP Done')
    
#     icpTransformFilter = vtk.vtkTransformPolyDataFilter()
#     icpTransformFilter.SetInputData(m1)
#     icpTransformFilter.SetTransform(icp)
#     icpTransformFilter.Update()
    
#     transformedSource = icpTransformFilter.GetOutput()
    
#     distance = vtkHausdorffDistancePointSetFilter()
#     distance.SetInputData(0, transformedSource)
#     distance.SetInputData(1, m2)
#     distance.Update()
    
#     print('Distance computation Done')
#     distanceAfterAlign = distance.GetOutput(0).GetFieldData().GetArray('HausdorffDistance').GetComponent(0, 0)
    
#     if distanceAfterAlign < best_transform:
#         best_transform = icp
#         best_value = distanceAfterAlign
        
#     print(i, distanceAfterAlign)

156161 180796
(156161, 3) (180796, 3)
Points Created
ICP Done
Points Created
ICP Done
Points Created
ICP Done
Points Created
ICP Done
Points Created
ICP Done
Points Created
ICP Done
Points Created
ICP Done
Points Created
ICP Done
Points Created
ICP Done
Points Created
ICP Done


2022-04-09 13:04:09.305 (  81.840s) [        BDD92180]vtkIterativeClosestPoin:273    ERR| vtkIterativeClosestPointTransform (0x5641ab64ba90): Can't execute with nullptr or empty input


In [28]:
vp1.

SyntaxError: invalid syntax (4032310572.py, line 1)

In [ ]:
vp1 = vtk.vtkPoints()
vp1.SetData(p1)

In [ ]:
a = vtk.vtkPolyDataReader()
#a.SetFileName('/home/pranjal.sahu/SlicerMorph/data/A_J_skull_.vtk')
a.SetFileName('movingMesh.vtk')
a.Update()
m1 = a.GetOutput()

a = vtk.vtkPolyDataReader()
#a.SetFileName('/home/pranjal.sahu/SlicerMorph/data/Ark_J_skull_.vtk')
a.SetFileName('fixedMesh.vtk')
a.Update()
m2 = a.GetOutput()

icp = vtk.vtkIterativeClosestPointTransform()
icp.SetSource(m1)
icp.SetTarget(m2)
icp.GetLandmarkTransform().SetModeToRigidBody()
icp.DebugOn()
icp.SetDebug(True)
icp.CheckMeanDistanceOn()
icp.SetMaximumNumberOfLandmarks(500)
icp.SetMaximumNumberOfIterations(10)
icp.SetMaximumMeanDistance(0.001)
icp.StartByMatchingCentroidsOn()
icp.Modified()
icp.Update()

icpTransformFilter = vtk.vtkTransformPolyDataFilter()
icpTransformFilter.SetInputData(m1)
icpTransformFilter.SetTransform(icp)
icpTransformFilter.Update()

transformedSource = icpTransformFilter.GetOutput()

t1 = vtk.vtkPolyDataWriter()
t1.SetInputData(transformedSource)
t1.SetFileName('transformedSource.vtk')
t1.Update()

itkwidgets.view(geometries=[m1, m2, transformedSource])

In [ ]:
def readvtk(filename):
    a = vtk.vtkPolyDataReader()
    a.SetFileName(filename)
    a.Update()
    m1 = a.GetOutput()
    return m1

a1 = readvtk('/home/pranjal.sahu/SlicerMorph/data/A_J_skull_.vtk')
a2 = readvtk('/home/pranjal.sahu/SlicerMorph/data/Ark_J_skull_.vtk')

print(a1.GetNumberOfPoints(), a2.GetNumberOfPoints())

In [ ]:
b2 = readvtk('fixedMesh.vtk')
b1 = readvtk('movingMesh.vtk')

print(b1.GetNumberOfPoints(), b2.GetNumberOfPoints())

In [ ]:
itkwidgets.view(geometries=[a1, b1, a2, b2])

In [ ]:
# TSD registration

imageDiagonal   = 50
stretchWeight   = 0.0001
bendWeight      = 0.0001
geometricWeight = 0.0001
maxStep         = 0.01
numberOfIterations = 10

print('Number of Points are')
print(movingMesh.GetNumberOfPoints())
print(fixedMesh.GetNumberOfPoints())

fixedMesh.BuildCellLinks()
movingMesh.BuildCellLinks()

PixelType = itk.D
Dimension = 3

MeshType        = itk.Mesh[itk.D, Dimension]
FixedImageType  = itk.Image[PixelType, Dimension]

# For getting the Bounding Box
ElementIdentifierType = itk.UL
CoordType = itk.F
Dimension = 3

VecContType = itk.VectorContainer[
    ElementIdentifierType, itk.Point[CoordType, Dimension]
]
bounding_box = itk.BoundingBox[ElementIdentifierType, Dimension, CoordType, VecContType].New()
bounding_box.SetPoints(movingMesh.GetPoints())
bounding_box.ComputeBoundingBox()

minBounds = np.array(bounding_box.GetMinimum())
maxBounds = np.array(bounding_box.GetMaximum())


spacing = np.sqrt(bounding_box.GetDiagonalLength2()) / imageDiagonal
diff = maxBounds - minBounds

print('Spacing ', spacing)
print('minBounds ', minBounds)
print('maxBounds ', maxBounds)

fixedImageSize    = [0]*3
fixedImageSize[0] = math.ceil( 1.2 * diff[0] / spacing )
fixedImageSize[1] = math.ceil( 1.2 * diff[1] / spacing )
fixedImageSize[2] = math.ceil( 1.2 * diff[2] / spacing )

fixedImageOrigin    = [0]*3
fixedImageOrigin[0] = minBounds[0] - 0.1 * diff[0]
fixedImageOrigin[1] = minBounds[1] - 0.1 * diff[1]
fixedImageOrigin[2] = minBounds[2] - 0.1 * diff[2]

fixedImageSpacing   = np.ones(3)*spacing
fixedImageDirection = np.identity(3)


fixedImage = FixedImageType.New()
fixedImage.SetRegions(fixedImageSize)
fixedImage.SetOrigin( fixedImageOrigin )
fixedImage.SetDirection( fixedImageDirection )
fixedImage.SetSpacing( fixedImageSpacing )
fixedImage.Allocate()


# Create BSpline Transformation object and initialize the parameters
SplineOrder = 3
TransformType  = itk.BSplineTransform[itk.D, Dimension, SplineOrder]
InitializerType = itk.BSplineTransformInitializer[TransformType, FixedImageType]

transform = TransformType.New()

numberOfGridNodesInOneDimension = 5
transformInitializer = InitializerType.New()
transformInitializer.SetTransform(transform)
transformInitializer.SetImage(fixedImage)
transformInitializer.SetTransformDomainMeshSize(numberOfGridNodesInOneDimension - SplineOrder)
transformInitializer.InitializeTransform()



MetricType = itk.ThinShellDemonsMetricv4.MD3
metric = MetricType.New()
metric.SetStretchWeight(stretchWeight)
metric.SetBendWeight(bendWeight)
metric.SetGeometricFeatureWeight(geometricWeight)
metric.UseConfidenceWeightingOn()
metric.UseMaximalDistanceConfidenceSigmaOff()
metric.UpdateFeatureMatchingAtEachIterationOn()
metric.SetMovingTransform(transform)
# Reversed due to using points instead of an image
# to keep semantics the same as in itkThinShellDemonsTest.cxx
# For the ThinShellDemonsMetricv4 the fixed mesh is regularized
metric.SetFixedPointSet(movingMesh)
metric.SetMovingPointSet(fixedMesh)
metric.SetVirtualDomainFromImage(fixedImage)
metric.Initialize()

print('TSD Metric Created')

shiftScaleEstimator = itk.RegistrationParameterScalesFromPhysicalShift[MetricType].New()
shiftScaleEstimator.SetMetric(metric)
shiftScaleEstimator.SetVirtualDomainPointSet(metric.GetVirtualTransformedPointSet())


optimizer = itk.ConjugateGradientLineSearchOptimizerv4Template.D.New()
optimizer.SetNumberOfIterations( numberOfIterations )
optimizer.SetScalesEstimator( shiftScaleEstimator )
optimizer.SetMaximumStepSizeInPhysicalUnits( maxStep )
optimizer.SetMinimumConvergenceValue( 0.0 )
optimizer.SetConvergenceWindowSize( numberOfIterations )

def iteration_update():
    metric_value = optimizer.GetValue()
    current_parameters = optimizer.GetCurrentPosition()
    print(f"Metric: {metric_value:.8g}")

iteration_command = itk.PyCommand.New()
iteration_command.SetCommandCallable(iteration_update)
optimizer.AddObserver(itk.IterationEvent(), iteration_command)


print('Number of Transform Parameters ', transform.GetNumberOfParameters())

AffineRegistrationType = itk.ImageRegistrationMethodv4.REGv4D3D3TD3D3MD3.New()
registration = AffineRegistrationType.New()
registration.SetNumberOfLevels(1)
registration.SetObjectName("registration")
registration.SetFixedPointSet(movingMesh)
registration.SetMovingPointSet(fixedMesh)
registration.SetInitialTransform(transform)
registration.SetMetric(metric)
registration.SetOptimizer(optimizer)
registration.InPlaceOn()

print('Registration Object created')
print('Initial Value of Metric ', metric.GetValue())

try:
    registration.Update()
except e:
    print('Error is ', e)

print('Final Value of TSD Metric ', metric.GetValue())


# Get the Displacement field from the BSpline Transform
finalTransform = registration.GetModifiableTransform()
convertFilter = itk.TransformToDisplacementFieldFilter.IVF33D.New()
convertFilter.SetTransform(finalTransform)
convertFilter.UseReferenceImageOn()
convertFilter.SetReferenceImage(fixedImage)
convertFilter.Update()
field = convertFilter.GetOutput()
field = np.array(field)
np.save('displacement_field.npy', field)

e_metric = itk.EuclideanDistancePointSetToPointSetMetricv4.PSD3.New()
e_metric.SetFixedPointSet(fixedMesh)
e_metric.SetMovingPointSet(movingMesh)
print('Euclidean Metric Before TSD Deformable Registration ', e_metric.GetValue())

numberOfPoints = movingMesh.GetNumberOfPoints()
for n in range(0, numberOfPoints):
    movingMesh.SetPoint(n, finalTransform.TransformPoint(movingMesh.GetPoint(n)))

e_metric = itk.EuclideanDistancePointSetToPointSetMetricv4.PSD3.New()
e_metric.SetFixedPointSet(fixedMesh)
e_metric.SetMovingPointSet(movingMesh)
print('Euclidean Metric After TSD TSD Deformable Registration ', e_metric.GetValue())

itk.meshwrite(movingMesh, 'result_bspline.vtk')

In [ ]:
metric.GetValue()

In [ ]:
# Visualize the fixed mesh, moving mesh and the registered mesh

a = vtk.vtkPolyDataReader()
a.SetFileName('movingMesh.vtk')
a.Update()
m1 = a.GetOutput()

a = vtk.vtkPolyDataReader()
a.SetFileName('/data/ITKPR22/ITK/ITK-build/Wrapping/Generators/Python/result_bspline.vtk')
a.Update()
m2 = a.GetOutput()

a = vtk.vtkPolyDataReader()
a.SetFileName('fixedMesh.vtk')
a.Update()
m3 = a.GetOutput()

itkwidgets.view(geometries=[m1, m2, m3])

In [ ]:
# Visualized the Displacement Field obtained from BSpline Transform

import matplotlib.pyplot as plt
plt.imshow(field[15, :, :, 0], cmap='gray')